# Topological Blankets: From Energy Landscapes to Markov Blanket Structure

This notebook demonstrates the **Topological Blankets (TB)** method, a geometric approach to discovering discrete Markov blanket structure from continuous energy landscape geometry.

The key insight: *every energy-based model defines a Riemannian manifold whose curvature encodes conditional independence.* By analyzing Hessian structure (second-order geometry of the energy surface), TB recovers which variables form coherent "objects" and which serve as statistical boundaries (Markov blankets) between them.

**Outline:**
1. Theory Overview
2. Synthetic Validation (Quadratic EBM)
3. Bridge Experiments (2D Score Model)
4. World Model Demo (LunarLander Active Inference + Dreamer)
5. Multi-Scale Comparison
6. Edge-Compute Implications

In [ ]:
import sys
import os
import json
import numpy as np
import matplotlib.pyplot as plt
from pathlib import Path
import warnings
warnings.filterwarnings('ignore')

# Project root
ROOT = Path(os.path.abspath('')).parent
sys.path.insert(0, str(ROOT))

RESULTS = ROOT / 'results'
FIGURES = ROOT / 'paper' / 'figures'

# Plotting style
plt.rcParams.update({
    'figure.figsize': (10, 6),
    'font.size': 11,
    'axes.grid': True,
    'grid.alpha': 0.3,
})

print(f'Project root: {ROOT}')
print(f'Results directory: {RESULTS}')
print(f'Available result files: {len(list(RESULTS.glob("*.json")))}')

---
## 1. Theory Overview

### What is a Markov Blanket?

A **Markov blanket** is the minimal set of variables that renders a target variable conditionally independent of all others. In a Gaussian graphical model with precision matrix $\Theta$, the blanket of variable $i$ is exactly $\{j : \Theta_{ij} \neq 0\}$.

### The Energy Landscape Connection

For an energy-based model $p(x) \propto e^{-E(x)}$, the Hessian of the energy encodes coupling structure:

$$H_{ij} = \frac{\partial^2 E}{\partial x_i \partial x_j}$$

If $H_{ij} = 0$, variables $i$ and $j$ are conditionally independent given the rest. TB estimates this Hessian from gradient samples and uses spectral analysis to discover block structure.

### The TB Pipeline

1. **Collect gradient samples**: Either compute $\nabla E(x)$ analytically or via autodiff
2. **Estimate the Hessian**: $\hat{H} = \text{Cov}(\nabla E)$ (gradient covariance)
3. **Normalize coupling**: $C_{ij} = |H_{ij}| / \sqrt{H_{ii} H_{jj}}$ (correlation-like)
4. **Detect blankets**: Separate variables into internal (object) and boundary (blanket) roles
5. **Cluster objects**: Spectral clustering on the internal-variable subgraph

The result is a partition of all variables into discrete objects separated by Markov blankets.

---
## 2. Synthetic Validation: Quadratic EBM

The simplest test case: a quadratic energy $E(x) = \frac{1}{2} x^T \Theta x$ with a known block-diagonal precision matrix. The ground truth partition is known exactly, so recovery can be measured precisely.

In [ ]:
from topological_blankets import TopologicalBlankets

# Build a block-structured precision matrix
# 2 objects (3 vars each) + 3 blanket variables = 9 total
n_objects = 2
vars_per_object = 3
vars_per_blanket = 3
intra_strength = 6.0    # Strong within-object coupling
blanket_strength = 0.8  # Weak cross-coupling via blanket

n_vars = n_objects * vars_per_object + vars_per_blanket
Theta = np.zeros((n_vars, n_vars))

# Strong within-object couplings
for i in range(n_objects):
    s = i * vars_per_object
    e = s + vars_per_object
    Theta[s:e, s:e] = intra_strength
    np.fill_diagonal(Theta[s:e, s:e], intra_strength * vars_per_object)

# Blanket block
b_start = n_objects * vars_per_object
Theta[b_start:, b_start:] = 1.0
np.fill_diagonal(Theta[b_start:, b_start:], vars_per_blanket)

# Weak cross-couplings through blanket only
for i in range(n_objects):
    s = i * vars_per_object
    e = s + vars_per_object
    Theta[s:e, b_start:] = blanket_strength
    Theta[b_start:, s:e] = blanket_strength

Theta = (Theta + Theta.T) / 2.0
eigvals = np.linalg.eigvalsh(Theta)
if eigvals.min() < 0.1:
    Theta += np.eye(n_vars) * (0.2 - eigvals.min())

# Ground truth
gt = np.full(n_vars, -1)  # -1 = blanket
for i in range(n_objects):
    gt[i*vars_per_object:(i+1)*vars_per_object] = i

print(f'Precision matrix shape: {Theta.shape}')
print(f'Ground truth: {gt}')
print(f'  Object 0: vars {list(np.where(gt==0)[0])}')
print(f'  Object 1: vars {list(np.where(gt==1)[0])}')
print(f'  Blanket:  vars {list(np.where(gt==-1)[0])}')

In [ ]:
# Sample via Langevin dynamics and collect gradients
np.random.seed(42)
n_samples = 5000
step_size = 0.005
temp = 0.1

samples, gradients = [], []
x = np.random.randn(n_vars)

for i in range(n_samples * 50):
    grad = Theta @ x
    noise = np.sqrt(2 * step_size * temp) * np.random.randn(n_vars)
    x = x - step_size * grad + noise
    if i % 50 == 0:
        samples.append(x.copy())
        gradients.append((Theta @ x).copy())

samples = np.array(samples)
gradients = np.array(gradients)
print(f'Collected {samples.shape[0]} samples with {gradients.shape[1]}-dim gradients')

In [ ]:
# Run Topological Blankets
tb = TopologicalBlankets(method='gradient', n_objects=2)
tb.fit(gradients)

objects = tb.get_objects()
blankets = tb.get_blankets()
coupling = tb.get_coupling_matrix()
assignment = tb.get_assignment()

print('TB Result:')
for obj_id, vars_idx in objects.items():
    print(f'  Object {obj_id}: vars {list(vars_idx)}')
print(f'  Blanket: vars {list(blankets)}')

# Metrics
from sklearn.metrics import adjusted_rand_score, f1_score
internal_mask = gt >= 0
ari = adjusted_rand_score(gt[internal_mask], assignment[internal_mask])
blanket_f1 = f1_score((gt == -1).astype(int), (assignment == -1).astype(int))
print(f'\nObject ARI: {ari:.3f}')
print(f'Blanket F1: {blanket_f1:.3f}')

In [ ]:
# Visualize coupling matrix and partition
fig, axes = plt.subplots(1, 3, figsize=(15, 4.5))

# Panel 1: Precision matrix (ground truth)
ax = axes[0]
im = ax.imshow(np.abs(Theta), cmap='YlOrRd', aspect='auto')
ax.set_title('Precision Matrix $|\\Theta|$ (Ground Truth)')
ax.set_xlabel('Variable j')
ax.set_ylabel('Variable i')
for b in [vars_per_object, 2*vars_per_object]:
    ax.axhline(y=b-0.5, color='white', linewidth=2, linestyle='--')
    ax.axvline(x=b-0.5, color='white', linewidth=2, linestyle='--')
plt.colorbar(im, ax=ax, shrink=0.8)

# Panel 2: TB coupling matrix
ax = axes[1]
im = ax.imshow(coupling, cmap='hot', aspect='auto')
ax.set_title('Estimated Coupling $C_{ij}$ (TB)')
ax.set_xlabel('Variable j')
ax.set_ylabel('Variable i')
for b in [vars_per_object, 2*vars_per_object]:
    ax.axhline(y=b-0.5, color='cyan', linewidth=2, linestyle='--')
    ax.axvline(x=b-0.5, color='cyan', linewidth=2, linestyle='--')
plt.colorbar(im, ax=ax, shrink=0.8)

# Panel 3: Partition comparison
ax = axes[2]
var_idx = np.arange(n_vars)
colors_gt = ['#3498db' if g==0 else '#e74c3c' if g==1 else '#2ecc71' for g in gt]
colors_tb = ['#3498db' if a==0 else '#e74c3c' if a==1 else '#2ecc71' for a in assignment]
bar_width = 0.35
ax.bar(var_idx - bar_width/2, np.ones(n_vars), bar_width, color=colors_gt, label='Ground Truth', alpha=0.7)
ax.bar(var_idx + bar_width/2, np.ones(n_vars), bar_width, color=colors_tb, label='TB Result', alpha=0.7)
ax.set_xlabel('Variable Index')
ax.set_title(f'Partition Comparison (ARI={ari:.2f}, F1={blanket_f1:.2f})')
ax.set_yticks([])
# Legend patches
from matplotlib.patches import Patch
legend_elements = [Patch(facecolor='#3498db', label='Object 0'),
                   Patch(facecolor='#e74c3c', label='Object 1'),
                   Patch(facecolor='#2ecc71', label='Blanket')]
ax.legend(handles=legend_elements, loc='upper right')

plt.tight_layout()
plt.show()

### Strength Sweep: How Robust is Recovery?

Precomputed results from sweeping the blanket coupling strength across 7 levels with 10 trials each. TB is compared against two baselines (DMBD-style variational role assignment and AXIOM-style mixture decomposition).

In [ ]:
# Load precomputed strength sweep
sweep_files = sorted(RESULTS.glob('*strength_sweep_10trials.json'))
if sweep_files:
    with open(sweep_files[-1]) as f:
        sweep_data = json.load(f)
    metrics = sweep_data['metrics']
    
    strengths = sorted([float(s) for s in metrics.keys()])
    
    fig, axes = plt.subplots(1, 2, figsize=(12, 5))
    
    methods = ['tc', 'dmbd', 'axiom']
    labels = ['Topological Blankets', 'DMBD-style', 'AXIOM-style']
    colors = ['#2ecc71', '#3498db', '#e74c3c']
    
    for ax, metric_key, ylabel, title in [
        (axes[0], 'mean_ari', 'Adjusted Rand Index', 'Object Partition Recovery'),
        (axes[1], 'mean_f1', 'F1 Score', 'Blanket Detection'),
    ]:
        for method, label, color in zip(methods, labels, colors):
            means = [metrics[str(s)][method][metric_key] for s in strengths]
            std_key = metric_key.replace('mean', 'std')
            stds = [metrics[str(s)][method][std_key] for s in strengths]
            ax.errorbar(strengths, means, yerr=stds, label=label,
                       color=color, marker='o', capsize=3)
        ax.set_xlabel('Blanket Strength (coupling)')
        ax.set_ylabel(ylabel)
        ax.set_title(title)
        ax.legend()
        ax.set_ylim(-0.1, 1.1)
    
    plt.tight_layout()
    plt.show()
    
    # Print summary at key strengths
    print('Summary at selected strengths:')
    for s in [0.1, 0.5, 0.8, 2.0]:
        sk = str(s)
        if sk in metrics:
            tc_ari = metrics[sk]['tc']['mean_ari']
            tc_f1 = metrics[sk]['tc']['mean_f1']
            print(f'  strength={s}: TB ARI={tc_ari:.3f}, F1={tc_f1:.3f}')
else:
    print('No strength sweep results found. Run experiments/quadratic_toy_comparison.py first.')

---
## 3. Bridge Experiment: 2D Score Model

Moving beyond closed-form energies, TB can analyze *learned* score functions. A small MLP is trained via denoising score matching on 2D synthetic datasets, and TB is applied to the learned score field.

This demonstrates TB's ability to extract structure from neural network energy landscapes rather than hand-crafted ones.

In [ ]:
# Load precomputed 2D score model results
score_files = sorted(RESULTS.glob('*score_model_2d.json'))
if score_files:
    with open(score_files[-1]) as f:
        score_data = json.load(f)
    
    print('2D Score Model Results')
    print('=' * 50)
    print(f'{"Dataset":<12s} {"Sample ARI":>10s} {"Blanket dims":>14s} {"Hessian diag":>14s}')
    print('-' * 50)
    
    for name, r in score_data['metrics'].items():
        H = np.array(r['hessian_est'])
        blanket = np.where(r['variable_blanket'])[0]
        print(f'{name:<12s} {r["sample_ari"]:>10.3f} {str(list(blanket)):>14s} '
              f'[{H[0,0]:.2f}, {H[1,1]:.2f}]')
    
    # Show the pre-generated figures
    score_figs = sorted(RESULTS.glob('*score_model_2d_score_field_moons.png'))
    if score_figs:
        from IPython.display import Image, display
        print('\nScore field and Hessian analysis for the moons dataset:')
        display(Image(filename=str(score_figs[-1]), width=700))
else:
    print('No score model results found. Run experiments/score_model_2d.py first.')

For 2D datasets, the Hessian is a $2 \times 2$ matrix. The off-diagonal coupling indicates how the two spatial dimensions interact under the energy landscape. Higher coupling suggests the dimensions cannot be factored apart; the score field has diagonal structure when the energy decomposes.

The sample-level ARI measures how well spectral clustering on score similarity recovers the true cluster labels. This is *not* the same as TB's variable-level analysis, but demonstrates that the learned score field captures meaningful structure.

---
## 4. World Model Demo: LunarLander-v3

The central application: applying TB to a trained world model's gradient landscape to discover which state variables form coherent subsystems.

### Setup

- **Environment**: LunarLander-v3 (8D state: x, y, vx, vy, angle, ang_vel, left_leg, right_leg)
- **Active Inference agent**: Ensemble of 5 MLPs predicting next-state distributions
- **Dreamer autoencoder**: Encoder (8D $\to$ 64D) + Decoder (64D $\to$ 8D), trained on reconstruction loss
- **Analysis**: TB applied to dynamics gradients in both 8D state space and 64D latent space

In [ ]:
# Load Active Inference TB analysis
actinf_files = sorted(RESULTS.glob('*actinf_tb_analysis.json'))
if actinf_files:
    with open(actinf_files[-1]) as f:
        actinf_data = json.load(f)
    
    state_labels = actinf_data['config']['state_labels']
    dynamics = actinf_data['metrics']['dynamics']
    
    print('Active Inference World Model: TB on 8D State Space')
    print('=' * 60)
    
    # Dynamics gradient assignment
    assign = dynamics['gradient_method']['assignment']
    is_blanket = dynamics['gradient_method']['is_blanket']
    
    obj0 = [state_labels[i] for i, a in enumerate(assign) if a == 0]
    obj1 = [state_labels[i] for i, a in enumerate(assign) if a == 1]
    blanket = [state_labels[i] for i, b in enumerate(is_blanket) if b]
    
    print(f'Dynamics gradient analysis:')
    print(f'  Object 0: {obj0}')
    print(f'  Object 1: {obj1}')
    print(f'  Blanket:  {blanket}')
    print(f'  Spectral eigengap: {dynamics["eigengap"]:.1f}')
    print(f'  Hierarchical levels: {len(dynamics["hierarchy"])}')
    
    # Coupling matrix visualization
    coupling = np.array(dynamics['coupling'])
    
    fig, axes = plt.subplots(1, 2, figsize=(13, 5))
    
    ax = axes[0]
    im = ax.imshow(coupling, cmap='hot', aspect='auto', vmin=0, vmax=1)
    ax.set_xticks(range(8))
    ax.set_xticklabels(state_labels, rotation=45, ha='right')
    ax.set_yticks(range(8))
    ax.set_yticklabels(state_labels)
    ax.set_title('Dynamics Coupling Matrix')
    plt.colorbar(im, ax=ax, shrink=0.8)
    
    # Color-code the partition
    ax = axes[1]
    colors = ['#3498db' if a==0 else '#e74c3c' if a==1 else '#2ecc71' for a in assign]
    grad_mag = dynamics['grad_magnitude']
    bars = ax.bar(range(8), grad_mag, color=colors)
    ax.set_xticks(range(8))
    ax.set_xticklabels(state_labels, rotation=45, ha='right')
    ax.set_ylabel('Mean $|\\nabla E|$')
    ax.set_title('Gradient Magnitude by Variable')
    legend_elements = [Patch(facecolor='#3498db', label='Object 0'),
                       Patch(facecolor='#e74c3c', label='Object 1'),
                       Patch(facecolor='#2ecc71', label='Blanket')]
    ax.legend(handles=legend_elements)
    
    plt.tight_layout()
    plt.show()
    
    print(f'\nPhysical interpretation:')
    print(f'  Object 0 ({obj0}): Position/contact subsystem')
    print(f'  Object 1 ({obj1}): Orientation/velocity subsystem')
    print(f'  Blanket ({blanket}): Angular velocity mediates between subsystems')
else:
    print('No Active Inference results found. Run experiments/world_model_analysis.py first.')

In [ ]:
# Multiple gradient landscapes
if actinf_files:
    landscapes = ['dynamics', 'reward', 'disagreement']
    landscape_labels = ['Dynamics', 'Reward', 'Ensemble Disagreement']
    
    print('Comparison Across Gradient Landscapes')
    print('=' * 70)
    print(f'{"Landscape":<24s} {"Object 0":<24s} {"Object 1":<24s} {"Blanket":<16s}')
    print('-' * 70)
    
    for lname, llabel in zip(landscapes, landscape_labels):
        data = actinf_data['metrics'][lname]
        a = data['gradient_method']['assignment']
        b = data['gradient_method']['is_blanket']
        o0 = [state_labels[i] for i, v in enumerate(a) if v == 0]
        o1 = [state_labels[i] for i, v in enumerate(a) if v == 1]
        bl = [state_labels[i] for i, v in enumerate(b) if v]
        print(f'{llabel:<24s} {str(o0):<24s} {str(o1):<24s} {str(bl):<16s}')
    
    print('\nDifferent landscapes reveal different structural views of the same system.')
    print('Dynamics: ang_vel as sole mediator between positional and orientational subsystems.')
    print('Disagreement: y and vy become blanket, reflecting where model uncertainty is highest.')

### Dreamer Autoencoder: TB in 64D Latent Space

A Dreamer-style autoencoder maps the 8D state to 64D latent codes. TB applied to reconstruction loss gradients in latent space discovers structure in the *learned representation*, not just the original state.

In [ ]:
# Load Dreamer TB analysis
dreamer_files = sorted(RESULTS.glob('*dreamer_tb_analysis.json'))
if dreamer_files:
    with open(dreamer_files[-1]) as f:
        dreamer_data = json.load(f)
    
    dm = dreamer_data['metrics']
    methods = dm.get('methods', dm)  # Handle both structures
    
    print('Dreamer Autoencoder: TB on 64D Latent Space')
    print('=' * 55)
    print(f'  Latent dimensions:    64')
    print(f'  Spectral eigengap:    {dm["eigengap"]:.1f}')
    
    # Gradient method results
    grad_m = methods.get('gradient', {})
    n_blanket_grad = grad_m.get('n_blanket', 0)
    obj_sizes_grad = grad_m.get('object_sizes', {})
    n_internal_grad = sum(obj_sizes_grad.values()) if isinstance(obj_sizes_grad, dict) else 0
    print(f'  Objects (gradient):   {len(obj_sizes_grad)} ({n_internal_grad} dims)')
    print(f'  Blanket (gradient):   {n_blanket_grad} dims')
    
    # Coupling method results
    coup_m = methods.get('coupling', {})
    n_blanket_coup = coup_m.get('n_blanket', 0)
    obj_sizes_coup = coup_m.get('object_sizes', {})
    n_internal_coup = sum(obj_sizes_coup.values()) if isinstance(obj_sizes_coup, dict) else 0
    print(f'  Objects (coupling):   {len(obj_sizes_coup)} ({n_internal_coup} dims)')
    print(f'  Blanket (coupling):   {n_blanket_coup} dims')
    
    hierarchy = dm.get('hierarchy', [])
    print(f'  Hierarchical levels:  {len(hierarchy)}')
    
    # Show Dreamer coupling and eigenvalue figures
    dreamer_coupling_figs = sorted(RESULTS.glob('*dreamer_analysis_dreamer_coupling_64d.png'))
    dreamer_eigen_figs = sorted(RESULTS.glob('*dreamer_analysis_dreamer_eigenvalue_spectrum.png'))
    
    figs_to_show = []
    if dreamer_coupling_figs:
        figs_to_show.append(('64D Latent Coupling Matrix', dreamer_coupling_figs[-1]))
    if dreamer_eigen_figs:
        figs_to_show.append(('Eigenvalue Spectrum', dreamer_eigen_figs[-1]))
    
    if figs_to_show:
        from IPython.display import Image, display
        for title, fpath in figs_to_show:
            print(f'\n{title}:')
            display(Image(filename=str(fpath), width=600))
else:
    print('No Dreamer results found. Run experiments/dreamer_analysis.py first.')

---
## 5. Multi-Scale Comparison

The same physical system (LunarLander) is analyzed at three representation levels. Do different representations agree on the underlying structure?

In [ ]:
# Load multi-scale comparison
ms_files = sorted(RESULTS.glob('*multi_scale_comparison.json'))
if ms_files:
    with open(ms_files[-1]) as f:
        ms_data = json.load(f)
    
    mm = ms_data['metrics']
    
    print('Multi-Scale Comparison: State Space vs Latent Space')
    print('=' * 60)
    
    # Comparison table
    print(f'{"Representation":<28s} {"Dims":>5s} {"Objects":>8s} {"Blanket":>8s} {"Eigengap":>9s}')
    print('-' * 60)
    for row in mm['comparison_table']:
        print(f'{row["name"]:<28s} {row["n_dims"]:>5d} {row["n_objects"]:>8d} '
              f'{row["n_blanket"]:>8d} {row["eigengap"]:>9.1f}')
    
    # Correspondence
    corr = mm['object_correspondence']
    nmi = corr['nmi_state_vs_projected_latent']
    
    print(f'\nNMI (state-space vs projected latent partition): {nmi:.3f}')
    print(f'\nState-space assignment: {corr["state_assignment"]}')
    print(f'Projected latent assignment: {corr["projected_latent_assignment"]}')
    
    # Latent-to-physical mapping
    print(f'\nLatent blanket maps most strongly to physical variables:')
    bp = corr['blanket_physical']
    for var, c in zip(bp['top_physical_vars'], bp['top_correlations']):
        print(f'  {var}: correlation = {c:.3f}')
    
    print(f'\n{mm["key_insight"]}')
    
    # Show multi-scale figures
    ms_figs = sorted(RESULTS.glob('*multi_scale_coupling_comparison.png'))
    if ms_figs:
        from IPython.display import Image, display
        print('\nCoupling matrices across representations:')
        display(Image(filename=str(ms_figs[-1]), width=700))
else:
    print('No multi-scale results found. Run experiments/multi_scale_comparison.py first.')

The NMI of ~0.52 indicates partial preservation of structure across representation levels. The latent space encodes the same physical system but distributes it across 64 dimensions, making the blanket structure more diffuse. This is expected: autoencoder latent codes entangle physical variables.

---
## 6. Edge-Compute Implications

If TB discovers that a system decomposes into $k$ objects of size $n_i$ with a blanket of size $n_b$, then inference can be *factored*: instead of inverting an $n \times n$ matrix (cost $\sim n^2$ for dense products), compute $k$ independent $n_i \times n_i$ problems plus a blanket update.

$$\text{Speedup} = \frac{n^2}{\sum_i n_i^2 + n_b^2 + k \cdot n_b \cdot \max(n_i)}$$

In [ ]:
# Load edge-compute analysis
edge_files = sorted(RESULTS.glob('*edge_compute_analysis.json'))
if edge_files:
    with open(edge_files[-1]) as f:
        edge_data = json.load(f)
    
    em = edge_data['metrics']
    
    print('Edge-Compute Factorization Savings')
    print('=' * 70)
    
    # Specific configurations
    configs = ['AI: 3+3+2b', 'Dreamer: 20+20+20+4b', 'Dreamer: 6x10+4b']
    print(f'{"Configuration":<25s} {"n":>5s} {"Monolithic":>11s} {"Factored":>10s} {"Speedup":>8s}')
    print('-' * 65)
    for cfg_name in configs:
        if cfg_name in em:
            c = em[cfg_name]
            print(f'{cfg_name:<25s} {c["n_total"]:>5d} {c["monolithic_flops"]:>11,d} '
                  f'{c["factored_flops"]:>10,d} {c["speedup"]:>7.1f}x')
    
    # Scaling behavior
    print(f'\nScaling with Dimension:')
    print(f'{"n_total":>8s} {"Monolithic":>12s} {"Factored":>12s} {"Speedup":>8s} {"Memory saved":>13s}')
    print('-' * 58)
    scaling = em.get('scaling', {})
    for dim_key in sorted(scaling.keys(), key=int):
        s = scaling[dim_key]
        mem_saved = 1.0 - s['factored_flops'] / s['monolithic_flops']
        print(f'{s["n_total"]:>8d} {s["monolithic_flops"]:>12,d} {s["factored_flops"]:>12,d} '
              f'{s["speedup"]:>7.1f}x {mem_saved:>12.1%}')
    
    # Speedup curve
    dims = [int(k) for k in sorted(scaling.keys(), key=int)]
    speedups = [scaling[str(d)]['speedup'] for d in dims]
    
    fig, ax = plt.subplots(figsize=(8, 5))
    ax.plot(dims, speedups, 'o-', color='#2ecc71', linewidth=2, markersize=8)
    ax.set_xlabel('State Dimension')
    ax.set_ylabel('Speedup Factor')
    ax.set_title('Factored Inference Speedup vs. Dimension')
    ax.set_xscale('log')
    ax.set_yscale('log')
    for d, s in zip(dims, speedups):
        ax.annotate(f'{s:.1f}x', (d, s), textcoords='offset points',
                   xytext=(0, 12), ha='center', fontsize=9)
    plt.tight_layout()
    plt.show()
    
    print(f'\nAt 4096 dimensions: {speedups[-1]:.1f}x speedup, '
          f'{1 - scaling[str(dims[-1])]["factored_flops"]/scaling[str(dims[-1])]["monolithic_flops"]:.0%} memory savings')
else:
    print('No edge-compute results found. Run experiments/edge_compute_analysis.py first.')

---
## Summary

Topological Blankets provides a *geometric* path from continuous energy landscapes to discrete Markov blanket structure:

| Experiment | Key Result |
|---|---|
| Synthetic (quadratic EBM) | ARI $\geq$ 0.95 when coupling ratio $> 2:1$ |
| 2D score model | Successfully extracts Hessian structure from learned score fields |
| LunarLander (8D state) | Discovers {position, contact} / {orientation, velocity} split with ang_vel as blanket |
| LunarLander (64D latent) | Eigengap=59.8, 40-dim object + 24-dim blanket |
| Multi-scale | NMI=0.52 between state-space and projected latent partition |
| Edge-compute | 25.9x speedup at 4096D, scaling log-linearly with dimension |

The approach requires only gradient samples, works with any differentiable energy function, and runs in seconds for practical dimensionalities. By factoring inference along discovered boundaries, it enables efficient deployment of world models on resource-constrained hardware.